In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# kinematics and dynamics code of these casadi objects can be found in https://github.com/edxmorgan/diff_uv project
# these can be replaced with your preferred uv dynamics
fb_id_func = cs.Function.load("float_base.casadi")
J_uv_func = cs.Function.load("J_uv.casadi")
uv_gn_func = cs.Function.load("gn.casadi") #z, eul, W, B, r_g, r_b

# ned jacobian for uv kinematics
J_uv = J_uv_func(base_ss.eul)

z_SX = base_ss.p_n[2]

# base bias force expressions
zero_acc = [0,0,0,0,0,0]
b_F = fb_id_func(z_SX, base_ss.eul, base_ss.v_base, zero_acc, base_ss.sim_p)

# base inverse dynamics expressions
tau_F = fb_id_func(z_SX, base_ss.eul, base_ss.v_base, base_ss.a_base, base_ss.sim_p)


use_coupled = True

# ned jacobian for uvms kinematics
J_uvms = cs.SX.eye(10)
J_uvms[:6,:6] = J_uv

J_uvms_func = cs.Function('J_uvms', [base_ss.eul], [J_uvms])
# J_uvms_func.save('J_uvms.casadi')

# c , cpp or matlab code generation for pid_control
# J_uvms_func.generate("J_uvms_func.c")
# os.system(f"gcc -fPIC -shared J_uvms_func.c -o libJ_uvms.so")

0